<a href="https://colab.research.google.com/github/deepmind/perception_test/blob/main/baselines/temporal_action_localisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure GPU runtime is enabled!!

In [ ]:
!git clone https://github.com/ptchallenge-workshop/actionformer_release_PT.git

Cloning into 'actionformer_release_PT'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 398 (delta 9), reused 0 (delta 0), pack-reused 373
Receiving objects: 100% (398/398), 639.31 KiB | 11.02 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [ ]:
%cd /content/actionformer_release_PT/libs/utils
!python setup.py install --user
%cd ../..

/content/actionformer_release_PT/libs/utils
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, p

In [ ]:
import os
import zipfile
import requests


def download_and_unzip(url: str, destination: str):
  """Downloads and unzips a .zip file to a destination.

  Downloads a file from the specified URL, saves it to the destination
  directory, and then extracts its contents.

  If the file is larger than 1GB, it will be downloaded in chunks,
  and the download progress will be displayed.

  Args:
    url (str): The URL of the file to download.
    destination (str): The destination directory to save the file and
      extract its contents.
  """
  if not os.path.exists(destination):
    os.makedirs(destination)

  filename = url.split('/')[-1]
  file_path = os.path.join(destination, filename)

  if os.path.exists(file_path):
    print(f'{filename} already exists. Skipping download.')
    return

  response = requests.get(url, stream=True)
  total_size = int(response.headers.get('content-length', 0))
  gb = 1024*1024*1024

  if total_size / gb > 1:
    print(f'{filename} is larger than 1GB, downloading in chunks')
    chunk_flag = True
    chunk_size = int(total_size/100)
  else:
    chunk_flag = False
    chunk_size = total_size

  with open(file_path, 'wb') as file:
    for chunk_idx, chunk in enumerate(
        response.iter_content(chunk_size=chunk_size)):
      if chunk:
        if chunk_flag:
          print(f"""{chunk_idx}% downloading
          {round((chunk_idx*chunk_size)/gb, 1)}GB
          / {round(total_size/gb, 1)}GB""")
        file.write(chunk)
  print(f"'{filename}' downloaded successfully.")

  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(destination)
  print(f"'{filename}' extracted successfully.")

  os.remove(file_path)

In [ ]:
!mkdir data
!mkdir data/pt

In [ ]:
# @title Download data
data_path = './data/pt'

train_annot_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_train_annotations.zip'
download_and_unzip(train_annot_url, data_path)
train_feat_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_train_video_features.zip'
download_and_unzip(train_feat_url, data_path)

valid_annot_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_valid_annotations.zip'
download_and_unzip(valid_annot_url, data_path)
valid_feat_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_valid_video_features.zip'
download_and_unzip(valid_feat_url, data_path)

'action_localisation_train_annotations.zip' downloaded successfully.
'action_localisation_train_annotations.zip' extracted successfully.
'action_localisation_train_video_features.zip' downloaded successfully.
'action_localisation_train_video_features.zip' extracted successfully.
'action_localisation_valid_annotations.zip' downloaded successfully.
'action_localisation_valid_annotations.zip' extracted successfully.
'action_localisation_valid_video_features.zip' downloaded successfully.
'action_localisation_valid_video_features.zip' extracted successfully.


In [ ]:
!python3 train.py configs/perception_video_train.yaml --output reproduce

2023-07-04 12:12:56.076110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 12:12:57.421898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
{'dataset': {'crop_ratio': [0.9, 1.0],
             'default_fps': 15,
             'downsample_rate': 1,
             'feat_folder': './data/pt/action_localisation_train_video_features',
             'feat_stride': 16,
             'file_ext': '.npy',
             'file_prefix': 'v_',
             'force_upsampling': True,
             'input_dim': 512,
             'input_modality': 'video',
             'json_file': './data/pt/action_localisation_train.json',
             'max_seq_len': 192,
             'mm_feat_folder': None,
             'num_classes':

In [ ]:
!python eval.py configs/perception_video_valid.yaml ckpt/perception_video_train_reproduce/

{'dataset': {'crop_ratio': [0.9, 1.0],
             'default_fps': 15,
             'downsample_rate': 1,
             'feat_folder': './data/pt/action_localisation_valid_video_features',
             'feat_stride': 16,
             'file_ext': '.npy',
             'file_prefix': 'v_',
             'force_upsampling': True,
             'input_dim': 512,
             'input_modality': 'video',
             'json_file': './data/pt/action_localisation_valid.json',
             'max_seq_len': 192,
             'mm_feat_folder': None,
             'num_classes': 63,
             'num_frames': 16,
             'trunc_thresh': 0.5},
 'dataset_name': 'perception',
 'devices': ['cuda:0'],
 'init_rand_seed': 1234567891,
 'loader': {'batch_size': 16, 'num_workers': 4},
 'model': {'backbone_arch': (2, 2, 5),
           'backbone_type': 'convTransformer',
           'embd_dim': 512,
           'embd_kernel_size': 3,
           'embd_with_ln': True,
           'fpn_dim': 512,
           'fpn_start_